# Evaluators Tutorial

**-- On evaluation --**

In this tutorial, we will cover different approaches to model evaluation. Let's start by investigating `perturb-lib` evaluators:

In [1]:
import perturb_lib as plib

plib.list_evaluators()

['MAE', 'Pearson', 'R2', 'RMSE']

Let us focus on standard RMSE evaluator for example:

In [2]:
evaluator_id = "RMSE"
evaluator = plib.load_evaluator(evaluator_id)
plib.describe_evaluator(evaluator_id)

'Root-mean-square error (RMSE).'

Let us now get datasets and train a sample model.

In [5]:
plib_data = plib.load_plibdata(contexts_ids=["HumanCellLine_K562_10xChromium3-scRNA-seq_Replogle22"])
traindata, _, testdata = plib.split_plibdata_3fold(
    plib_data, context_ids="HumanCellLine_K562_10xChromium3-scRNA-seq_Replogle22"
)
model = plib.load_model(
    "Catboost",
    model_args={
        "learning_rate": 0.75,
        "random_seed": 11,
        "iterations": 1000,
        "task_type": "CPU",
    },
)
model.fit(traindata)

16:10:50 | INFO | Loading training data into RAM..
16:10:51 | INFO | ReactomePathways.gmt.zip found in cache.
16:10:51 | INFO | Reducing embedding to 20 dimensions using PCA.
16:10:51 | INFO | Embedding training data of size 12344962..
16:10:51 | INFO | Creating X matrix
16:10:54 | INFO | Creating y vector
16:10:54 | INFO | Fitting CatBoostRegressor on data of shape (12344962, 41)..


0:	learn: 0.1748921	total: 742ms	remaining: 12m 21s
1:	learn: 0.1748209	total: 1.06s	remaining: 8m 50s
2:	learn: 0.1747386	total: 1.37s	remaining: 7m 35s
3:	learn: 0.1746807	total: 1.58s	remaining: 6m 32s
4:	learn: 0.1745931	total: 1.82s	remaining: 6m 1s
5:	learn: 0.1745169	total: 2.06s	remaining: 5m 42s
6:	learn: 0.1744569	total: 2.29s	remaining: 5m 24s
7:	learn: 0.1743925	total: 2.54s	remaining: 5m 15s
8:	learn: 0.1743312	total: 2.8s	remaining: 5m 8s
9:	learn: 0.1742843	total: 3.06s	remaining: 5m 2s
10:	learn: 0.1742344	total: 3.28s	remaining: 4m 55s
11:	learn: 0.1742010	total: 3.5s	remaining: 4m 48s
12:	learn: 0.1741531	total: 3.74s	remaining: 4m 44s
13:	learn: 0.1741033	total: 4.01s	remaining: 4m 42s
14:	learn: 0.1740492	total: 4.25s	remaining: 4m 39s
15:	learn: 0.1740027	total: 4.51s	remaining: 4m 37s
16:	learn: 0.1739472	total: 4.8s	remaining: 4m 37s
17:	learn: 0.1738952	total: 5.09s	remaining: 4m 37s
18:	learn: 0.1738460	total: 5.37s	remaining: 4m 37s
19:	learn: 0.1737908	total:

16:16:51 | INFO | Model fitting completed


999:	learn: 0.1698714	total: 4m 35s	remaining: 0us


We can now evaluate_model results as follows:

In [6]:
evaluator.evaluate(model.predict(testdata), testdata)

0.17545009567806363

Introducing new evaluators is also trivial:

In [6]:
@plib.register_evaluator
class CoolEvaluator(plib.PlibEvaluatorMixin):
    def evaluate(self, predictions, true_values, context_adata=None) -> float:
        return 0.0

In [7]:
"CoolEvaluator" in plib.list_evaluators()

True